In [18]:
import numpy as np
import scanpy as sc
import multigrate as mtg
import anndata as ad
import scipy
from scipy import sparse
import os
import matplotlib.pyplot as plt

In [19]:
results_path = "../results/multigrate/trimodal/coef_1000.0/"

## Load "reference" combined anndata and query anndatas

In [20]:
combined = ad.read_h5ad(results_path+"combined.h5ad")
rna_query = ad.read_h5ad("../pp_harm_data/rna_query-pp-harm-sub.h5ad")
adt_query = ad.read_h5ad("../pp_harm_data/adt_query-pp-harm-sub.h5ad")

loading data..


## Recreate "modality_lengths"

had to remove it to solve a bug with writing the concatenated anndata in the multigrate reference training

In [21]:
#had to delete that due to bug
rna = ad.read_h5ad("../pp_harm_data/rna-pp-harm-sub.h5ad")
adt = ad.read_h5ad("../pp_harm_data/adt-pp-harm-sub.h5ad")
cytof = ad.read_h5ad("../pp_harm_data/cytof-pp-harm-sub.h5ad")
intersection = adt.var.index.intersection(cytof.var.index).tolist()
print("adt and cytof have {} common features".format(len(intersection)))
adt_unique = adt[:, adt.var_names.drop(intersection)].copy()
cytof_unique = cytof[:, cytof.var_names.drop(intersection)].copy()
adt_common = adt[:, intersection].copy()
cytof_common = cytof[:, intersection].copy()
print("concatenating data..")
combined_uns = mtg.data.organize_multiome_anndatas(
    adatas = [[rna, None], [adt_common, cytof_common], [adt_unique, None], [None, cytof_unique]],
    layers = [[None, None], [None, None], [None, None], [None, None]],
)
combined.uns['modality_lengths'] = combined_uns.uns['modality_lengths'].copy()
combined.uns['modality_lengths']

adt and cytof have 26 common features
concatenating data..


/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


{0: 4000, 1: 26, 2: 166, 3: 22}

## Setup query anndatas

In [22]:
cytof_unique = cytof_unique[:0,:] #remove all observations from the cytof modalities
cytof_common = cytof_common[:0,:]

adt_query_unique = adt_query[:, adt_query.var_names.drop(intersection)].copy()
adt_query_common = adt_query[:, intersection].copy()
query_adata = mtg.data.organize_multiome_anndatas(
    adatas = [[rna_query, None], [adt_query_common, cytof_common], [adt_query_unique, None], [None, cytof_unique]],
    layers = [[None, None], [None, None], [None, None], [None, None]],
)
query_adata

/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:872: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    []

    Inferred to be: empty

  names = self._prep_dim_index(names, "obs")
/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:872: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    []

    Inferred to be: empty

  names = self._prep_dim_index(names, "obs")
/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 32980 × 4214
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC'

## Load model

In [23]:
model = mtg.model.MultiVAE.load(dir_path = results_path+'multigrate.dill', adata=combined, use_gpu='cuda:0')

INFO     File ../results/multigrate/trimodal/coef_1000.0/multigrate.dill/model.pt already downloaded               


/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:91: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [24]:
model.to_device('cuda:0')

## Prepare query anndata

In [25]:
mtg.model.MultiVAE.prepare_query_anndata(adata=query_adata, reference_model=model, return_reference_var_names=False, inplace=True)

INFO     Found 100.0% reference vars in query data.                                                                


## Load query data

In [29]:
mtg.model.MultiVAE.load_query_data(adata=query_adata, reference_model=model, use_gpu='cuda:0', freeze=True, ignore_categories=None)

/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:91: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


Training status: Not Trained